In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install openpyxl

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle as cPickle
import pandas as pd
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from keras.layers import LSTM, Dense, TimeDistributed, Bidirectional
import sklearn.metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras import Model
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional
from keras import regularizers
from keras import models
from openpyxl import Workbook

In [ ]:
path_data = "/content/drive/MyDrive/SuccKD_MultiSpecies/Data/"
path_model = "/content/drive/MyDrive/SuccKD_MultiSpecies/Model/"
path_result = "/content/drive/MyDrive/SuccKD_MultiSpecies/Result_cv/"

In [ ]:
file_train = "Train_H.sapiens_succ.csv"

In [ ]:
# Build an amino acid dictionary
def twoTupleDic1():
    AA_list_sort = ['G','A','V','L','I','M','P','F','W','S','T','N','Q','Y','C','K','R','H','D','E','X']

    AA_dict = {}
    numm = 1
    for i in AA_list_sort:
        AA_dict[i] = numm
        numm += 1
    return AA_dict
def ProSentence(pro, K):
	sentence = ""
	length = len(pro)
	for i in range(length - K + 1):
		sentence += pro[i: i + K] + " "
    #delete extra space
	sentence = sentence[0 : len(sentence) - 1]
	return sentence
k =1#1-gram
word_index1 = twoTupleDic1()
vocab_size = len(word_index1)


In [ ]:
#  Load train data and pre-processing
df_train =pd.read_csv(path_data+file_train, delimiter= ',')
texts_train =[]
for i in df_train['Peptide_WS_33']:
  temp = ProSentence(i,k)
  texts_train.append(temp)
df_train['k_mer'] =texts_train
train_sequences = []
for each in texts_train:
    each_index_list = []
    each = each.split(' ')
    for i in each:
        each_index_list.append(word_index1[i])
    train_sequences.append(each_index_list)
# Tokenizer
data_token = []
for i in df_train['k_mer']:
   data_token.append(i.split())
MAX_SEQUENCE_LENGTH = len(data_token[1])
Xtrain = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
ytrain = np.array(df_train['Label'])
ytrain = np.array(ytrain)
#one-hot encoding on the labels
lb = LabelBinarizer()
ytrain= lb.fit_transform(ytrain)
ytrain = to_categorical(ytrain)
ytrain.shape


(4053, 2)

In [ ]:
TIME_STEPS = MAX_SEQUENCE_LENGTH
INPUT_SIZE = 300

In [ ]:
# KD2_succi
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras import models, layers, losses

# Teacher 1
def create_cnn_model():
    model = models.Sequential()
    model.add(layers.Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True))
    model.add(layers.Dropout(0.4))
    model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Dropout(0.4))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))  # Binary classification (softmax for distillation)
    return model

# Teacher 2
def create_bilstm_model():
    model = models.Sequential()
    model.add(layers.Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True))
    model.add(layers.Bidirectional(layers.LSTM(units=32, return_sequences=True), input_shape=(TIME_STEPS, INPUT_SIZE)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.2))
    model.add(layers.Bidirectional(layers.LSTM(units=32, return_sequences=True), input_shape=(TIME_STEPS, INPUT_SIZE)))
    model.add(layers.BatchNormalization())
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))  # Binary classification (softmax for distillation)
    return model
# Student
def create_student():
    model = models.Sequential()
    model.add(layers.Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True))
    model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Dropout(0.2))
    model.add(layers.BatchNormalization())
    model.add(layers.Bidirectional(layers.LSTM(units=32, return_sequences=True), input_shape=(TIME_STEPS, INPUT_SIZE)))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))  # Binary classification (softmax for distillation)
    return model
#  KD2_succi: Knowlege distillation
import tensorflow as tf
from tensorflow.keras import optimizers

class Distiller(tf.keras.Model):
    def __init__(self, teacher1, teacher2, student, alpha=0.5, temperature=3.0):
        super(Distiller, self).__init__()
        self.teacher1 = teacher1
        self.teacher2 = teacher2
        self.student = student
        self.alpha = alpha
        self.temperature = temperature
        # Optimizer for the student
        self.student_optimizer = AdamW(learning_rate=0.0001)

    def compile(self, metrics, student_loss_fn, distillation_loss_fn):
        super(Distiller, self).compile(metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn

    def call(self, inputs):
        """
        This method defines how the model processes inputs.
        Args:
        - inputs: Input data (Xtest_student or any other data).

        Returns:
        - Output logits from the student model.
        """
        student_logits = self.student(inputs, training=False)
        return student_logits

    def train_step(self, data):
        x, y = data

        # Sequentially forward pass through teachers
        teacher1_logits = self.teacher1(x, training=False)
        teacher2_logits = self.teacher2(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass through student
            student_logits = self.student(x, training=True)

            # Compute the task loss (for hard labels)
            student_loss = self.student_loss_fn(y, student_logits)

            # Compute distillation losses (for soft labels)
            distill_loss1 = self.distillation_loss_fn(
                tf.nn.softmax(teacher1_logits / self.temperature, axis=1),
                tf.nn.softmax(student_logits / self.temperature, axis=1)
            ) * (self.temperature ** 2)

            distill_loss2 = self.distillation_loss_fn(
                tf.nn.softmax(teacher2_logits / self.temperature, axis=1),
                tf.nn.softmax(student_logits / self.temperature, axis=1)
            ) * (self.temperature ** 2)

            # Combined loss
            total_loss = self.alpha * student_loss + (1 - self.alpha) * (distill_loss1 + distill_loss2)

        # Compute gradients and update student model
        grads = tape.gradient(total_loss, self.student.trainable_variables)
        self.student_optimizer.apply_gradients(zip(grads, self.student.trainable_variables))

        # Update metrics for the student
        self.compiled_metrics.update_state(y, student_logits)

        # Return results
        results = {
            'student_loss': student_loss,
            'distill_loss1': distill_loss1,
            'distill_loss2': distill_loss2
        }
        results.update({m.name: m.result() for m in self.metrics})
        return results

    def test_step(self, data):
        x, y = data
        student_logits = self.student(x, training=False)

        # Compute the task loss (for hard labels)
        student_loss = self.student_loss_fn(y, student_logits)

        # Update metrics for the student
        self.compiled_metrics.update_state(y, student_logits)

        return {
            'student_loss': student_loss,
            **{m.name: m.result() for m in self.metrics}
        }



In [ ]:
# Transfer learning from Generic dl set to species
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import AdamW
from keras import models
k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)
all_metrics = []
confusion_matrices = []

# K-fold cross-validation
for fold, (train_index, val_index) in enumerate(kf.split(Xtrain)):
    print(f"Training on fold {fold + 1}/{k}...")
    X_train_fold, X_val_fold = Xtrain[train_index], Xtrain[val_index]
    y_train_fold, y_val_fold = ytrain[train_index], ytrain[val_index]
    student_fold = create_student()
    teacher1_fold = models.load_model(path_model +"model_teacher21.h5") # Sử dụng teacher giống nhau trên cùng loài
    teacher2_fold = models.load_model(path_model +"model_teacher22.h5")
    distiller_fold = Distiller(teacher1=teacher1_fold, teacher2=teacher2_fold, student=student_fold, alpha=0.5, temperature=10)
    distiller_fold.compile(
        student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),

        metrics=["categorical_accuracy", "AUC"],
        distillation_loss_fn=keras.losses.KLDivergence(),
    )

    # Train model
    history = distiller_fold.fit(
        X_train_fold, y_train_fold,
        validation_data=(X_val_fold, y_val_fold),
        batch_size=16,
        epochs=50,

    )

    results = distiller_fold.evaluate(X_val_fold, y_val_fold)
    all_metrics.append(results)
    y_pred_fold = student_fold.predict(X_val_fold)
    y_pred_classes = np.argmax(y_pred_fold, axis=1)
    y_true_classes = np.argmax(y_val_fold, axis=1)

    # confusion matrix and save TP, TN, FP, FN
    cm = confusion_matrix(y_true_classes, y_pred_classes)
    tn, fp, fn, tp = cm.ravel()
    confusion_matrices.append({"TP": tp, "TN": tn, "FP": fp, "FN": fn})
    print(f"Fold {fold + 1} - TP: {tp}, TN: {tn}, FP: {fp}, FN: {fn}")

# Average the results from the folds
avg_metrics = np.mean(all_metrics, axis=0)
print(f"\nAverage Metrics over {k}-Fold Cross Validation:")
for metric_name, avg_metric in zip(distiller_fold.metrics_names, avg_metrics):
    print(f"{metric_name}: {avg_metric:.4f}")

#  Show TP, TN, FP, FN
for fold, cm in enumerate(confusion_matrices):
    print(f"Fold {fold + 1} - TP: {cm['TP']}, TN: {cm['TN']}, FP: {cm['FP']}, FN: {cm['FN']}")
# save TP, TN, FP, FN into file Excel
df_confusion = pd.DataFrame(confusion_matrices)
df_confusion.to_excel(path_result + "cv_H.Sapiens_KD2_Succi.xlsx", index=False)

print("\nConfusion matrix results saved to 'cv_H.Sapiens_KD2_Succi.xlsx'")

Training on fold 1/10...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:567: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:582: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


228/228 ━━━━━━━━━━━━━━━━━━━━ 17s 29ms/step - AUC: 0.6493 - categorical_accuracy: 0.6117 - distill_loss1: 0.0320 - distill_loss2: 0.0817 - loss: 0.5000 - student_loss: 0.6366 - val_AUC: 0.7300 - val_categorical_accuracy: 0.6478 - val_loss: 0.5000 - val_student_loss: 0.6084
Epoch 2/50
228/228 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - AUC: 0.7527 - categorical_accuracy: 0.6816 - distill_loss1: 0.0293 - distill_loss2: 0.0726 - loss: 0.5000 - student_loss: 0.5915 - val_AUC: 0.7701 - val_categorical_accuracy: 0.6847 - val_loss: 0.5000 - val_student_loss: 0.5468
Epoch 3/50
228/228 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - AUC: 0.7729 - categorical_accuracy: 0.7081 - distill_loss1: 0.0263 - distill_loss2: 0.0663 - loss: 0.5000 - student_loss: 0.5587 - val_AUC: 0.7948 - val_categorical_accuracy: 0.7365 - val_loss: 0.5000 - val_student_loss: 0.4459
Epoch 4/50
228/228 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - AUC: 0.8116 - categorical_accuracy: 0.7350 - distill_loss1: 0.0237 - distill_loss2: 0.0615 - loss: 0.5000 

...

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Fold 9 - TP: 89, TN: 232, FP: 33, FN: 51
Training on fold 10/10...


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:567: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:582: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


228/228 ━━━━━━━━━━━━━━━━━━━━ 13s 25ms/step - AUC: 0.6222 - categorical_accuracy: 0.5903 - distill_loss1: 0.0327 - distill_loss2: 0.0845 - loss: 0.5000 - student_loss: 0.6493 - val_AUC: 0.7689 - val_categorical_accuracy: 0.7136 - val_loss: 0.5000 - val_student_loss: 0.5670
Epoch 2/50
228/228 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - AUC: 0.7198 - categorical_accuracy: 0.6612 - distill_loss1: 0.0297 - distill_loss2: 0.0752 - loss: 0.5000 - student_loss: 0.6088 - val_AUC: 0.7770 - val_categorical_accuracy: 0.7086 - val_loss: 0.5000 - val_student_loss: 0.4745
Epoch 3/50
228/228 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - AUC: 0.7555 - categorical_accuracy: 0.6836 - distill_loss1: 0.0283 - distill_loss2: 0.0707 - loss: 0.5000 - student_loss: 0.5814 - val_AUC: 0.7843 - val_categorical_accuracy: 0.7136 - val_loss: 0.5000 - val_student_loss: 0.3860
Epoch 4/50
228/228 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - AUC: 0.7896 - categorical_accuracy: 0.7154 - distill_loss1: 0.0260 - distill_loss2: 0.0654 - loss: 0.5000 

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import AdamW

k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)
all_metrics = []
confusion_matrices = []
# K-fold cross-validation
for fold, (train_index, val_index) in enumerate(kf.split(Xtrain)):
    print(f"Training on fold {fold + 1}/{k}...")
    X_train_fold, X_val_fold = Xtrain[train_index], Xtrain[val_index]
    y_train_fold, y_val_fold = ytrain[train_index], ytrain[val_index]
    teacher1_fold = create_cnn_model()
    teacher2_fold = create_bilstm_model()
    student_fold = create_student()
    teacher1_fold.compile(
        loss=keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=[
            "categorical_accuracy",
            "AUC",
        ],
        optimizer=keras.optimizers.AdamW(learning_rate=0.0001),
    )
    teacher2_fold.compile(
        loss=keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=[
            "categorical_accuracy",
            "AUC",
        ],
        optimizer=keras.optimizers.AdamW(learning_rate=0.0001),
    )
    teacher1_fold.fit(
        X_train_fold, y_train_fold,
        batch_size =16,
        epochs=50
    )

    teacher2_fold.fit(
        X_train_fold, y_train_fold,
        batch_size =16,
        epochs=50
    )
    # Re-initialize the distiller for each fold
    distiller_fold = Distiller(teacher1=teacher1_fold, teacher2=teacher2_fold, student=student_fold, alpha=0.5, temperature=10)
    distiller_fold.compile(
        student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),

        metrics=["categorical_accuracy", "AUC"],
        distillation_loss_fn=keras.losses.KLDivergence(),
    )

    # Train model
    history = distiller_fold.fit(
        X_train_fold, y_train_fold,
        validation_data=(X_val_fold, y_val_fold),
        batch_size=16,
        epochs=50,
        verbose=1
    )

     # Save result of each fold
    results = distiller_fold.evaluate(X_val_fold, y_val_fold)
    all_metrics.append(results)
    y_pred_fold = student_fold.predict(X_val_fold)
    y_pred_classes = np.argmax(y_pred_fold, axis=1)
    y_true_classes = np.argmax(y_val_fold, axis=1)

    # confusion matrix and save TP, TN, FP, FN
    cm = confusion_matrix(y_true_classes, y_pred_classes)
    tn, fp, fn, tp = cm.ravel()
    confusion_matrices.append({"TP": tp, "TN": tn, "FP": fp, "FN": fn})
    print(f"Fold {fold + 1} - TP: {tp}, TN: {tn}, FP: {fp}, FN: {fn}")

# Average the results from the folds
avg_metrics = np.mean(all_metrics, axis=0)
print(f"\nAverage Metrics over {k}-Fold Cross Validation:")
for metric_name, avg_metric in zip(distiller_fold.metrics_names, avg_metrics):
    print(f"{metric_name}: {avg_metric:.4f}")

# Show TP, TN, FP, FN for each fold
for fold, cm in enumerate(confusion_matrices):
    print(f"Fold {fold + 1} - TP: {cm['TP']}, TN: {cm['TN']}, FP: {cm['FP']}, FN: {cm['FN']}")
# save TP, TN, FP, FN into the file Excel
df_confusion = pd.DataFrame(confusion_matrices)
df_confusion.to_excel(path_result + "cv_H.Sapiens_KD2_Succ1.xlsx", index=False)

print("\nConfusion matrix results saved to 'cv_H.Sapiens_KD2_Succ1.xlsx'")

Training on fold 1/10...
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5575: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


228/228 [==============================] - 3s 6ms/step - loss: 0.7575 - categorical_accuracy: 0.5945 - auc: 0.6232
Epoch 2/50
228/228 [==============================] - 1s 6ms/step - loss: 0.7151 - categorical_accuracy: 0.6082 - auc: 0.6382
Epoch 3/50
228/228 [==============================] - 1s 6ms/step - loss: 0.6790 - categorical_accuracy: 0.6301 - auc: 0.6644
Epoch 4/50
228/228 [==============================] - 1s 6ms/step - loss: 0.6611 - categorical_accuracy: 0.6400 - auc: 0.6780
Epoch 5/50
228/228 [==============================] - 1s 6ms/step - loss: 0.6610 - categorical_accuracy: 0.6403 - auc: 0.6752
Epoch 6/50
228/228 [==============================] - 1s 5ms/step - loss: 0.6398 - categorical_accuracy: 0.6438 - auc: 0.6926
Epoch 7/50
228/228 [==============================] - 1s 5ms/step - loss: 0.6407 - categorical_accuracy: 0.6526 - auc: 0.6964
Epoch 8/50
228/228 [==============================] - 1s 5ms/step - loss: 0.6326 - categorical_accuracy: 0.6674 - auc: 0.7043
Epo

/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5575: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


228/228 [==============================] - 3s 6ms/step - loss: 0.7562 - categorical_accuracy: 0.5747 - auc: 0.6065
Epoch 2/50
228/228 [==============================] - 1s 6ms/step - loss: 0.7108 - categorical_accuracy: 0.6112 - auc: 0.6437
Epoch 3/50
228/228 [==============================] - 1s 6ms/step - loss: 0.6911 - categorical_accuracy: 0.6104 - auc: 0.6498
Epoch 4/50
228/228 [==============================] - 1s 6ms/step - loss: 0.6634 - categorical_accuracy: 0.6318 - auc: 0.6739
Epoch 5/50
228/228 [==============================] - 1s 6ms/step - loss: 0.6586 - categorical_accuracy: 0.6361 - auc: 0.6750
Epoch 6/50
228/228 [==============================] - 1s 6ms/step - loss: 0.6481 - categorical_accuracy: 0.6509 - auc: 0.6885
Epoch 7/50
228/228 [==============================] - 1s 6ms/step - loss: 0.6408 - categorical_accuracy: 0.6452 - auc: 0.6952
Epoch 8/50
228/228 [==============================] - 1s 6ms/step - loss: 0.6323 - categorical_accuracy: 0.6559 - auc: 0.7022
Epo

...

228/228 [==============================] - 3s 5ms/step - loss: 0.7558 - categorical_accuracy: 0.5943 - auc: 0.6217
Epoch 2/50
228/228 [==============================] - 1s 5ms/step - loss: 0.6983 - categorical_accuracy: 0.6099 - auc: 0.6531
Epoch 3/50
228/228 [==============================] - 1s 5ms/step - loss: 0.6829 - categorical_accuracy: 0.6280 - auc: 0.6640
Epoch 4/50
228/228 [==============================] - 1s 5ms/step - loss: 0.6634 - categorical_accuracy: 0.6409 - auc: 0.6754
Epoch 5/50
228/228 [==============================] - 1s 5ms/step - loss: 0.6528 - categorical_accuracy: 0.6450 - auc: 0.6831
Epoch 6/50
228/228 [==============================] - 1s 5ms/step - loss: 0.6397 - categorical_accuracy: 0.6565 - auc: 0.6965
Epoch 7/50
228/228 [==============================] - 1s 5ms/step - loss: 0.6420 - categorical_accuracy: 0.6623 - auc: 0.6957
Epoch 8/50
228/228 [==============================] - 1s 5ms/step - loss: 0.6320 - categorical_accuracy: 0.6604 - auc: 0.7034
Epo

/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5575: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


228/228 [==============================] - 3s 6ms/step - loss: 0.8029 - categorical_accuracy: 0.5573 - auc: 0.5659
Epoch 2/50
228/228 [==============================] - 1s 5ms/step - loss: 0.6950 - categorical_accuracy: 0.6135 - auc: 0.6468
Epoch 3/50
228/228 [==============================] - 1s 5ms/step - loss: 0.6715 - categorical_accuracy: 0.6294 - auc: 0.6638
Epoch 4/50
228/228 [==============================] - 1s 5ms/step - loss: 0.6599 - categorical_accuracy: 0.6373 - auc: 0.6714
Epoch 5/50
228/228 [==============================] - 1s 5ms/step - loss: 0.6549 - categorical_accuracy: 0.6395 - auc: 0.6744
Epoch 6/50
228/228 [==============================] - 1s 5ms/step - loss: 0.6404 - categorical_accuracy: 0.6541 - auc: 0.6944
Epoch 7/50
228/228 [==============================] - 1s 5ms/step - loss: 0.6409 - categorical_accuracy: 0.6494 - auc: 0.6897
Epoch 8/50
228/228 [==============================] - 1s 5ms/step - loss: 0.6275 - categorical_accuracy: 0.6708 - auc: 0.7065
Epo

/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5575: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


228/228 [==============================] - 3s 5ms/step - loss: 0.7322 - categorical_accuracy: 0.6075 - auc: 0.6383
Epoch 2/50
228/228 [==============================] - 1s 5ms/step - loss: 0.7130 - categorical_accuracy: 0.6075 - auc: 0.6368
Epoch 3/50
228/228 [==============================] - 1s 5ms/step - loss: 0.6834 - categorical_accuracy: 0.6223 - auc: 0.6569
Epoch 4/50
228/228 [==============================] - 1s 5ms/step - loss: 0.6627 - categorical_accuracy: 0.6338 - auc: 0.6734
Epoch 5/50
228/228 [==============================] - 1s 5ms/step - loss: 0.6507 - categorical_accuracy: 0.6475 - auc: 0.6861
Epoch 6/50
228/228 [==============================] - 1s 5ms/step - loss: 0.6459 - categorical_accuracy: 0.6428 - auc: 0.6876
Epoch 7/50
228/228 [==============================] - 1s 5ms/step - loss: 0.6440 - categorical_accuracy: 0.6499 - auc: 0.6921
Epoch 8/50
228/228 [==============================] - 1s 5ms/step - loss: 0.6317 - categorical_accuracy: 0.6617 - auc: 0.7054
Epo